# EY AA COE OCR POST PROCESSING LOGIC 
## TEXT NAVIGATION TEMPLATE 
### Version: AA COE INTERNAL         
##### Date Creation: 10/15
##### Last Edited: 10/5
##### Authors: Zack Carideo + Ran Zhou    

### Stage 1: Preprocess ,  OCR, and Transform Original Documents
    #### 1. Identify if PDF is searchable or unsearchable 
    #### 2. Repair , Clean and enhance original pdf(with image) 
    #### 3. Use LSTMs + OSD configuration from Tesseract to OCR PDF and Create searchable document 
    #### 4. Convert and Save searchable PDF into Formatted Text file preserving all spacing and formatting of original pdf 
    #### 5. Load Formated txt file and store each row individually into a list 

### Stage 2: Attribute Extraction 
    #### 1. Create Dictonary of Reg-ex patterns to identify attribute and table locations
    #### 2. Search rows in doc for a match to a reg-ex pattern associated with attribute in question (row, startpos, endpos)
    #### 3. Parse attribute value from text based on loc relationship with attribute header and apply Checks to filter out FPs
            # 1. Check text to right of header (first word > 5 characters, first 2 words are not text, text contains num) 
            # 2. if no text found to right , check text BELOW attribute header using header loc as starting pos for next line 
    #### 4. Apply check to ensure correct attribute extracted (check:  1 group of text with  numbers, chars,  special chars
    
### Stage 3: Table Extraction 
    #### 1. Find starting row of table using regex patterns stored in Dictonary that indicate table start 
    #### 2. Find ending location of table using patterns associateed with end of table for a particular company 
    #### 3. Extract information between start and end row of table using delimiter(double space, works 80% of time)
    #### 3. For ~100% Accuracy, simply define character ranges that are associated with the table for each comapany
    
    
### NOTE: The below guide provides 3 templates for processing text around an "anchor point"(The header we are looking for) within a text file. 3 Key attributes(PO Number, Shipping Adderss, Table Data) from Purchase Orders have been used as examples to illistrate how each template is used.  

### Import Modules

In [1]:
import os 
import re 
from collections import OrderedDict
import operator 
from operator import itemgetter
import stat
import sys, getopt
from subprocess import Popen, PIPE
import subprocess
from re import finditer
import fileinput
from statistics import mode 

#table extraction 
from prettytable import PrettyTable
import pandas as pd
import numpy as np


### DEFINE INPUTS 

In [2]:
##################
###FILE IMPORT####
#####INPUTS#######

#1.Location of all text files converted using AA COE OCR Function 
#txt_path = "//home//docadmin//python//ocr//samples//Converted_txtfiles//"
txt_path="C://Users//caridza//Desktop//pythonScripts//OCR//THE SOLUTION//PriorGen_OCRSolution//ServerBackUpCodeRepo//txt_output//"

#2.File number you want to process(variable or needs to be an iterable object)
filenum =11

#3.Delimiter used for loading filefile is delimited (defaults to "\n")
delim = "\n"

#4.Encoding used (if special characters are present you must use utf_8_sig)
encoding = "utf_8_sig"

#Attribute being Processed 
Attribute = 'SHIPTO'


####################################
##NAVIGATION SPECIFICATION INPUTS###
####################################

##
#CHECKS:SEARCHING / RETURNING TEST ON MULTIPLE ROWS AROUND(ABOVE/BELOW and RIGHT)HEADER
## 

#1.DETERMINE LOCATION OF TARGET TEXT:characters to the right of a header that are checked when determining if 
#the text associated with the header is to right or below the header.
CHECK_Space2RightHeader = 8

#2.IDENTIFY RANGE OF CHARACTERS TO SCAN ON EACH LINE BELOW HEADER
#NOTE: Used in determining the range of characters on below lines to consider in excess of the range of the attribute header 
#NOTE: Only used when target text is identified to be BELOW header
CHECK_Txt2Right = 10
CHECK_Txt2Left = 2

#3.IDENTIFY ROW BELOW HEADER TO START LOOKING FOR BLANK ROWS
#Note: Start row below a header to begin searching for presence of space or another Attribute header
#Note: 2 is used becuase sometimes headers have 2 blank lines below before information begins
#Note: Only used when target text is identified to be BELOW header 
EndRowCheck_StartBelow = 2

#4.MAX ROWS TO CHECK BELOW ATTRIBUTE HEADER  
#Note: Checking for presence of a Space or another Attribute Header, if either are found 
#      we extract the address information up until this line. 
#Note: 10 is used to account for addresses with blank lines in between each entry 
#Note: if blank line or attribute heaer found prior to the max line defined below, the loop stops at that line. 
EndRowCheck_EndBelow = 10 

#5.IDENTIFY TOTAL LINES ABOVE/BELOW HEADER TO CONSIDER when target text is to the RIGHT of the attribute header 
#Note: Only used when target text is identified to RIGHT of header
CHECK_LinesAbove = 5
CHECK_LinesBelow = 6

##
#CHECKS:SEARCHING / RETURNING SINGLE LINE OF INFO TO RIGHT OR BELOW HEADER 
## 

#1.DEFINE MINIMUM THRESHOLDS FOR DETERMINING IF STRING TO RIGHT OF REG-EX MATCH IS VALID
#Note: 5 is used specifically for PO # and will not be valid for other attributes. 
#Note: these therasholds must be set based on the 'expected minimum valid' length of attribute expected. 
#Min length of entire string to right of matched result
Words2Right = 5       
#Min length of the first word in string to right of matched result 
MinWordCharacters = 5 

#2.DEFINE TOTAL NUMBER OF EXPECTED WORDS TO THE RIGHT OF THE MATCHED HEADER WE EXPECT THE ATTRIBUTE RESULTS TO HAVE
#NOTE: For PO # we only expect 1 word , for atributes like ADDRESS, this would need to be increased to refelct n multiple words
ExpectedWrdsToRight = 0 #indicates how many words to the right of the attribute header should be considered in the final value (0 = 1)

#3.DEFINE the maximum difference in the length of the line where match was found and the lines below this line 
#Note: This is only used in situations where attribute is found BELOW header
#Note: This control ensures the line below the header has a similar length to the line the header was found on
percentdiflength =.05

#4.DEFINE VALIDATION TEST
#NOTE: This test is applied to the result found as a final check point to ensure the format of the returned results is inline with 
#      the format of the expected response
ValidationTest = re.compile(r'(.*[a-zA-Z])?(.*[0-9])(.*[!-@#$%&*])?')

#5.DEFINE EXPANSION FACTOR
#this is the fraction of the total length of the below the line a match was found on that the script will use 
#to increase the total range of characters considered when looking for attributes below a specific attribute header
ExpansionFactor = 20   



############
###INPUTS###
####END#####

### Upload Text File to Parse 

In [6]:
#Generates list of all txt files in the directory where converted text files are stored 
files = getfiles(txt_path, ".txt")

#Open file 
f =open(txt_path+files[filenum],encoding = encoding)

#Read file as string
a= f.read()

#Split lines into list based on new line indicator
b = a.split(delim)

#remove all completely white lines (helps postprocessing and identifying values that are below associated attribute)
c = [x for x in b if x]

print(c)

['Cooper Tire & Rubber Co.                                                              CHANGE PURCHASE ORDER: 4400111503', 'To: Ge Intelligent Platforms Inc              Ship To: Caryn Sutherland                       Order Date: 04/19/2017        Delivery Date: 06/30/2018', '    PO Box 8106                                         Cooper Tire and Rubber Co             Ship Via:', '    Charlottesville, VA US 22906-8106                   701 Lima Avenue                       Payment Terms: Net 75', 'Vendor#: 7090004525                                     Findlay, OH US                        Currency: USD', 'Contact:                                                45840-3215', "Phone: 800-433-2682                           Buyer's Name: Rusty Aldrich", 'Email: gef.psb@gefanuc.com                    Phone: 419-423-1321', '                                              Fax:', '                                              Email: rsaldrich@coopertire.com', '                 MATERIAL         

### Create Dictonary to hold all patterns used to identify attribute names 

#### GDS Note: The below template provides a guideline for creating a dictonary of key value pairs to find "PO Number" ,  identify the starting poisition of a table, and generate control checkpoints to ensure the string found matches the format of the attribute. This dictonary must be expanded to include a key value pair for all attributes in scope. 

#### 1. Each attribute that must be found requires a key:value pair in the below dictonary 
##### (Key,Value): Key = Name of attribute we are looking for , Value = Regex patterns used to identify the attribute 
#### 2. Each control / validation check for each attribute requires a key value pair in the below dictonary 
##### (Key,Value): Key = Name of control we are applying , Value = Regex patterns used to validate the string returned has correct format/length 
#### 3.Each COMPANY will require unique END of table key:value pair in the dicontary. Currently we have illistrated how to use identify the start and end of table for GREYMATTER, similar approach must be taken for each company 
#### 4. As multiple attributes are added GDS may need multiple dictonaries(or dictonary of dictonaries), 1 to hold key/value attribute search rules, 1 to hold rules to validate results 

In [7]:
#create empty dictonary to hold key value pairs(attribute : regex)
#Note:holds all attributes and the regex used to find each attribute across all POs(except for tables, which are company specifc)
LookupTable = {}
        
#CONTROLS 
LookupTable['MultipleWordTest']=re.compile(r'^([a-zA-Z]+\s){2}') #Ensure PO number returned is not 2 full non integer words

#ATTRIBUTES
LookupTable['PO NUMBER']=re.compile(r' Order Number:|CHANGE PURCHASE ORDER:|\s{4}(P/O NUMBER)|P.O. NO.:|P.O. #:|PO #|PO Number|PO Number:| Purchase Order No:(#)?|PURCHASE ORDER(#)?')
LookupTable['SHIPTO']=re.compile(r'\s{4}Ship To(?!:)|Ship-to Address:|Ship To\s{4}|Ship To:|\s{4}SHIP(?! VIA)(?! TO)|Shipping Address|Ship-to-Address|\s{4}SHIP TO:|Shipping Adderss\s{4}|Ship/Service To:\s{4}|\s{4}SEND TO:')
LookupTable['COMPANY']=re.compile(r'AL HOSN|CB Pacific|BRIDGESTONE|Volvo|Cooper Tire|Gray Matter|FERMI|Gexpro|INTERTAPE POLYMER CORP|KC Global|Lafarge|DESCO|Sabritas')
LookupTable['TERMSPAY'] =re.compile(r'Terms of Payment|\s{4}TERMS\s{4}|^Terms')

#TABLE(start line) 
LookupTable['TABLE']=re.compile(r'Item #+( +PARTICULAR)(.*)|\s+ITEM+( +P. REQ.)(.*)|\s+ITEM+( +QTY)+( +UM)(.*)|^Item+( +Material/Service)+( Description)(.*)|\s+Line+( +Quantity)+( +Item)(.*)|\s+LN+( +ORDER)+( +QTY)(.*)|^Item+( +Quantity)(.*)|^Line+( +Item)(.*)|^Item Nr+( +Description)(.*)')

# ATTRIBUTE: TABLE(identify END line)
# Note: This is comapny specific, i have shown for Grey Matters Inc below. 
# Note:Based on pattern in company PO that can be used to identify End Of Table

# GREY MATTERS
# Note: Table ends at the line before the string "TOTAL" or "SUBTOTAL"(2 distinct layouts identifeid for GreyMatter)
LookupTable['GREYTABLEEND1']=re.compile(r'(?<!\S)Subtotal(?!\S)')  #Subtotal logic
LookupTable['GREYTABLEEND2']=re.compile(r'(^|\s{4,})Total(\s+|$)') #Total Logic (at least 4 spaces before the word being searched for)

### Store Match information for the attribute in question into a new dictonary 
#### Note: Identifies alll matches found for attribute in question and returns the line #, charter start/end position and the value the reg-ex matched on into a list. This informaiton is used to identify the starting location within the document that we will use to find the 'value' of the attribute header we are searching for.  

In [8]:
#CREATE EMPTY DICTONARY TO HOLD KEY VALUE PAIRS RETURNED FOR EACH ATTRIBUTE(contains all matches for the element in question)
#Key = ATtribute Name , Value = Locational info of match
test ={}

#CREATE EMPTY LIST TO HOLD INTERMEDIATE MATCH RESULTS(compiled within the FOR loop) 
#values in this list are passed to the dictonary "test" after the loop has covered all rows in document 
list1 = []

#FOR LOOP DESCRIPTION
#for every line in the PO look for pattern in text representative of the reg-ex patterns defined in the LookUpTable Dictonary
#and store each match found to a list. Once all matches have been stored to the list, assign each match as a key:value pair in the dicontary test
for i in range(len(c)):
   
    # apply regex pattern associated with attribute in question 
    # and for each pattern match found in line i generate locational information 
    for match in finditer(LookupTable[Attribute], c[i]):
     
        #ASSIGN DICTONARY VALUES FOR THE KEY BEING PROCESSED 
        #output nested list with tuple of locational information about the match 
        #Note:creates a 3 element tuple with the information on locaiton of string
        list1.append(([str(i)],[match.span()], [match.group()]))

#if no matches found raise a flag as the value of the tuple
if not list1: 
    NoMatchFound = 1 
else: 
    NoMatchFound = 0
    test[Attribute] = list1 

#Final Output: Matches and associated info tied to location of attribute 
for i in test.keys():
    for j in test[i]:
        print(j)

(['1'], [(46, 54)], ['Ship To:'])


# Process All Matches and Return Predicted Value

## SYNTAX USED FOR NAVIGATION OF DOCUMENT 
### c: list containing all text for each row in the original pdf
        - c[0] = first line of document 
        - c[j[0][0]] = line of document match was found on 
        - c[j[0][0]+1] = line below line in which the match was found on 
        
### j: contains locational information for the match being analyzed     
        - j[0][0]  = Line in document c in which the match was found (c[j[0][0]] returns all text in row  match was found on)
        - j[1][0] = start and end position of the characters the reg-ex pattern found a match on 
        - j[2][0] = the actual string of words from line c[j[0][0]] the reg-ex pattern matched on 

### Parsing Multi-line Text Feilds below or to the right of a specified key word 

In [17]:
#########EXAMPLE##########
###PARSING SHIP TO########
########ADDRESS###########

#CHECK match was found for attribute
if NoMatchFound ==0: 
    
    #create empty list to hold match details 
    matchdetails = []
    
    #inialize MatchNum which acts as control to prevent re-doing steps if match was found in prior iteration
    MatchNum = 0

    #create empty list to hold the final rows determined to be an address 
    StartPosBelowStore = []
        
# look through each match stored in "test" dictonary(from prior step) and determine 
# if the expected attribute value is to the right or below the attribute header/name
    for i in test.keys():
        for j in test[i]:

    ################################################
    #1.Generate Aggregated details on matched string  
    ################################################
            matchdetails.append(
                                   ( int(j[0][0])                             #Line in text inwhich the match was found 
                                   , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                                   , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                                   , j[1][0]                                  #Start and end position of the matched pattern found 
                                   , j[2][0]                                  #Actual value the pattern matched on 
                                   ))
    ###########################################################################################################
    # 2.PROCESS MATCH BASED ON LOCATION OF TEXT AROUND HEADER(BELOW or RIGHT)
    ###########################################################################################################
        #PROCESS 1: APPLICABLE WHEN TARGET TEXT IS BELOW HEADER
            #CHECK
            #1.Match results have been populated for the attribute being processed 
            #2.There is NOT text to the right of the "SHIP TO" header
            #3.Check for blank space to riht of  the input string 
            #4.No Valid match returned from prior match processed 
            if list(test.keys())[0] == Attribute \
            and re.search(re.compile(r'^\s*$'),c[int(j[0][0])][j[1][0][1]:j[1][0][1]+ CHECK_Space2RightHeader]) is not None \
            and StartPosBelowStore == []:
        
                #IDENTIFY IF BLANK LINES found after the 1st line below header(to determine ending line of address)
                #OR if line is not blank, CHECK TO SEE IF AN ATTRIBUTE HEADER IS PRESENT WHICH INDICATES NEW FEILD ON NEXT LINE
                #Note: we start at line 2 because we want to avoid finding a blank line if it is directly under the header(aka the address starts after one blank line below header)
                for i in range(EndRowCheck_StartBelow,EndRowCheck_EndBelow):

                    #string on line i
                    stringy = c[int(j[0][0])+i]
     
                    #Search for first word in below lines using expanded area based on header area
                    #max(0,startloc): if the start loc is at char <2 then we want to make sure we just pull back the string starting at 0
                    #print('2.look at region below header to identify start position of first word: ' +  stringy[max(0,j[1][0][0]-2):j[1][0][1]+10])
                    Range2Search = stringy[max(0,j[1][0][0]-CHECK_Txt2Left):j[1][0][1]+CHECK_Txt2Right]
                    
                    #CRITERA TO DETERIME BLANK / EOL FOR ADDRESS 
                    #1.No Match exists from prior loop 
                    #2.No Attribute header is on the next line(TABLE used as example) OR Next line is blank
                    #Note:Assigns the line associated with the  first occurance of blank space or attribute heaer to object break out of loop after first match and only process if a match in prior loop was not found
                    if MatchNum ==0 and(re.search(LookupTable['TABLE'],stringy) is not None \
                                        or re.search(LookupTable['TERMSPAY'],stringy) is not None \
                                        or re.search(LookupTable['PO NUMBER'],stringy) is not None \
                                        or re.search(re.compile(r'^\s*$'),Range2Search) is not None):     
                        
                        #line at which any of above conditions where met
                        #set MatchNum = 1 to indicate to other processes we already have a match 
                        #break out of loop after the first match has been identified
                        line = int(j[0][0])+i 
                        MatchNum = 1
                        break 
                        
                    #if neither scenerio is met, assign line to be the EndRowCheck_EndBElow value
                    #and print warning to screen
                    else: 
                        line = int(j[0][0])+i
                        print('CONTROL: Blank Line NOT Found(last line to consider defaults to 6th line below header): '+ str(line))
                
                #IDENTIFY START POSITION OF FIRST WORD ON EACH LINE BELOW HEADER 
                #Note: This will be used as our critera for determineing weather or not a line is part of the address
                #Note: Total Lines in Scope = line-int(j[0][0])+1 
                for i in range(1,line-int(j[0][0])+1):
                    
                    #string on ith line below or above line hit was found
                    stringy = c[int(j[0][0])+i]               
                    
                    #identify range in which we will search for the first word 
                    Range2Search = stringy[max(0,j[1][0][0]- CHECK_Txt2Left):j[1][0][1] + CHECK_Txt2Right] 
                    
                    #if the line below header has at least 2 words present then pull back the first word to use in identifying uniform starting positions
                    if  len(Range2Search.strip().split()) >0 : 
                        FirstwordBelow = Range2Search.strip().split()[0]
                        StartPosBelow= stringy.find(FirstwordBelow) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)   
                        StartPosBelowStore.append(StartPosBelow) #stores starting position of each first word considered on each row below the header
                                                
                #EXTRACT MODAL STARTING POSITION FROM THE LINES BELOW HEADER
                Mode = mode(StartPosBelowStore)
                     
                #Empty List to hold lines associated with Send To Address
                InScopeLines = []
                
                #loop through lines again and this time extract those with the modal starting position  
                for i in range(1,line-int(j[0][0])):
                    stringy = c[int(j[0][0])+i]              
                    Range2Search = stringy[max(0,j[1][0][0]-CHECK_Txt2Left):j[1][0][1]+CHECK_Txt2Right]  
                    
                    #if the line below header has at least 2 words present then 
                    #pull back the first word to use in identifying uniform starting positions
                    if  len(Range2Search.strip().split()) >0 : 
                        FirstwordBelow = Range2Search.strip().split()[0]
                        StartPosBelow= stringy.find(FirstwordBelow) 
                        
                        #if the starting position of the first word on the line below the header is within 1 char of the modal length 
                        #then append to results StartWrd2Right-1 <= 
                        if StartPosBelow-1 <= Mode<= StartPosBelow+1:
                            InScopeLines.append(stringy[StartPosBelow:len(stringy)].strip().split('    ')[0])
   
                            
    #  PROCESS USED IF ATTRIBUTE IS TO THE RIGHT OF HEADER 
        #  match results have been populated for(This should always be true) 
        #  AND check to make sure there is text to the right of the "SHIP TO" header
        
            if list(test.keys())[0] == Attribute \
            and re.search(re.compile(r'^\s*$'),c[int(j[0][0])][j[1][0][1]:j[1][0][1]+10]) is None:  #ensures there is not blank space to the right fo the header 
            
                # A.Subset document c , retaining on the text from the row the match was found on
                string = c[int(j[0][0])]
                
                # B.Identify the first character of first word to right of header
                    # Syntax breakdown
                        # Result1 = string.find(j[2][0]) # find starting position of the pattern the algo returned (the attribute header 
                        # Result2 = string[Result1+len(j[2][0]):len(string)] # return the portion of the string to the right of the header found
                        # Result3 = str(Result2.strip().split()[0]) #the first word to right of header  
                        # StartWrd2Right = string.find(Result3) #find the starting position of the word to the right of header on the line the header was found 
                StartWrd2Right = string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
              
            #C.Identify Controls (not actually used, just providing some ideas)
                
                #Total Spaces to Right of header before first word 
                #goal: prevent pulling in a sentence of text (that has SHIP in the middle)
                #Space2Right = StartWrd2Right - j[1][0][1]

                #Total Words to right of header
                #Words2Right =  wordCount(stripwhite(string[string.find(j[2][0])+len(j[2][0]):len(string)]))
                
                #Total Groups of words to right of  header 
                #Groups2Right =  wordCountDelim(stripwhite(string[string.find(j[2][0])+len(j[2][0]):len(string)]),delim="   ")
         
                #C. Find All linse below and above the current line that have a word starting in same position 
                InScopeLines = []
                for i in range(-min( int(j[0][0]) ,CHECK_LinesAbove),CHECK_LinesBelow):
                    
                    #string on ith line below or above line hit was found
                    stringy = c[int(j[0][0])+i]
                   
                    #starting position of the word on the next line at position right of header  
                    StartWrd2Righty = stringy.find(str(stringy[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
                    
                    #find if there any any words directly to left of word starting in position inlined with above/below liens
                    #if line being processed contains attribute header then look left of the attribute header, not the 
                    #first word to left of header
                    if re.search(LookupTable[Attribute],c[int(j[0][0])+i]) is not None: 
                        
                        #if its the row with the header, we dont want to include the header in the check for empty space
                        Space2Left = stringy[string.find(j[2][0])-5:string.find(j[2][0])]
                    else:
                        #look 5 characters back from the start of the first word to right of header
                        #these should all be blank if they are associated with the header as we see multiple blanks to left
                        #of ship to addresses where header is above text 
                        Space2Left = stringy[StartWrd2Righty-5:StartWrd2Righty]
                    
                    #if the 5 characters before the first word to right of header are blank then process
                    if re.search(re.compile(r'^\s*$'),Space2Left) is not None: 
         
                        #Text on below /above lines starting from the position of the first word to right of the header
                        #  NOTE: Start/Stop characters are determined by the position of the first word to the right of the 
                        #        attribbute haeder" from the orignial "string" (returns a range of characters to use in subsetting the full line)
                        #        the string of data that is subset using this range of characters is the string i rows above/below the row the header 
                        #        was found
                        NextLineToRight= stringy[string.find(j[2][0])+len(j[2][0]):len(string)]

                        #if the line below or above starts within 1 characters of the above line append 
                        if StartWrd2Right-1 <= StartWrd2Righty <= StartWrd2Right+1: 
                            InScopeLines.append(stringy[StartWrd2Right:len(stringy)].strip().split('    ')[0])


##FINAL OUTPUT 
print(InScopeLines)

['Caryn Sutherland', 'Cooper Tire and Rubber Co', '701 Lima Avenue', 'Findlay, OH US', '45840-3215']


## PARSING A SINGLE LINE TO RIGHT OR BELOW HEADER 

In [ ]:
#Create empty string to hold result
#First: We try to populate EndRes(the predicted value) using characters from line j[0][0] that are to the right of the attribute header the regex match was found at 
        # ToRight_Orig = string[string.find(j[2][0])+len(j[2][0]):len(string)] ==> Explanation ==> starting position of match + num characters associated with attribute being searched for : eol
#Second:If no relevant text is found to the right of the attribute header the regex matched on, we look to the lines below the header until we find the value we are intersted in 
EndRes = ""

#NOTE: This is a reg-ex pattern that creates a test to ensure the format of the 
#      response is in line with the format of a general response for the attribute in question
#Create Empty list to hold Desriptive sttatistics about line in which matches were found 
#Note: This is just for reference, and the resulting list is not used in any post processing 
matchdetails = []



#look through each match stored in "test" dictonary(from prior step) and determine 
#if the expected attribute value is to the right or below the attribute header/name
for i in test.keys():
    for j in test[i]:
        
        #1.Aggregated details on matched strings  
        matchdetails.append(
                               ( int(j[0][0])                             #Line in text inwhich the match was found 
                               , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                               , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                               , j[1][0]                                  #Start and end position of the matched pattern found 
                               , j[2][0]                                  #Actual value the pattern matched on 
                               ))

        #2.processing each match to find true result if the attribute being processed "ATTRIBUTE" is the attribute match results have been populated for(This should always be true)
        if list(test.keys())[0] == Attribute:
            
            #A.Subset document c , retaining on the text from the row the match was found on
            string = c[int(j[0][0])]
            print(string)
            
            #B.Identify information to the right of Attribute Name that match was found for 
            ToRight_Orig = string[string.find(j[2][0])+len(j[2][0]):len(string)] #starting position of match + num characters associated with attribute being searched for : eol
            ToRight_Strip = string[string.find(j[2][0])+len(j[2][0]):len(string)].strip() #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
            print(ToRight_Orig)
            
            #C.First Check if information to the right of Attribute Name is present / valid for a PO #
               #Note: The below checks ensures no null values are passed or sequences of words are passed to the PO Number Match Logic 
                 #1.length of entire string to right of matched result > 5 characters in length
                 #2.length of the first word in string to right of matched result > 5 characters in length  (this will be the value we pull for PO)
                 #3.String does not contain more than 2 consequtive words at bengingin of text to right of attribute found
            if len(ToRight_Strip)> Words2Right \
            and len(ToRight_Strip.split()[0])> MinWordCharacters \
            and re.search(LookupTable['MultipleWordTest'],ToRight_Strip) is None:  
                
                #IF ABOVE CONDITIONS MET, TAKE FIRST FIRST FOUND TO RIGHT OF ATTRIBUTE HEADER
                #Note: we are taking the first word because PO# is not a multi-word attribute
                       #for attributes with many words this should be changed to refelct the expected number of words. 
                if(ExpectedWrdsToRight >0):
                    nextword = ToRight_Strip.split()[:ExpectedWrdsToRight]
                else:
                    nextword = ToRight_Strip.split()[0]
                print(nextword)
            #D.RESULTS VALIDATION: for each match apply the following regex check: 
                #1.find an optional letter with anything before it
                #2.a number with anything before it
                #3.and an optional special character with anything before it
                for match in finditer(ValidationTest, nextword):
                    print(match.group())
                    EndRes = match.group() #if result is found at match i , this is populated and prevents further matches from being considered
                    print(EndRes)
        
    #3.If you cannot find any matches when looking to the right , look down at the lines below the attribute header
    if EndRes =="": 
        
        #initial value for number of lines to move down from original Attribute found(DO NOT CHANGE)
        itr=1 
        
        #loop through each match again
        for j in test[i]:
          
            #continue to move down the lines 1 at a time until the length of the orig + itr line is similar to the length 
            #of the line the attribute was found at. Becuase our text layout is preserved, the line with informaiton of PO should 
            #be of simlar length to the line the attribute header was pulled from 
            #NOTE: When comparing line lengths, we do not use the striped line, we want to preserve all leading and trailing spaces
            if len(c[int(j[0][0])+1]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*percentdiflength),0):
                
                #continue to loop until the length of line j[0][0]+itr is NOT less than .95 the length of the line the initial match was found on 
                #c[int(j[0][0])+itr]: the itr'th line below the line the match was found on 
                while len(c[int(j[0][0])+itr]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*percentdiflength),0):
                    
                    #if line j[0][0] + 1 does not meet above condition, we dont need to evalute it, so we place iterator at the front of the loop 
                    itr = itr + 1
                    
                    #text in itr'th line below the line the initial match was found on 
                    nextstring = c[int(j[0][0])+itr]
                    print('A',nextstring)
                    
                #else:#if the next line is of similar length to the line above, u dont need to use while loop, just take row i+1
                #    nextstring = c[int(j[0][0])+itr]
                    #print('B',nextstring)
            else: 
                #else:#if the next line is of similar length to the line above, u dont need to use while loop, just take row i+1
                nextstring = c[int(j[0][0])+itr]
                
            #remove white space from string satisfying all above conditions    
            nextstring_strip = nextstring.strip()
            print(nextstring)
            
            #length of next string
            Length = len(nextstring)
            
            #inialize itrerator to be used in expanding the range of linespace we pull back. this is done 
            #to ensure that we pull back one complete word 
            #NOTE: if you need to pull back multiple words then u need to change the ExpectedWrdsToRight variable 
            itr=2 #initialize i
             
            #if there are 0 words picked up in the text below(if its just blank space below)
            if wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < ExpectedWrdsToRight+1:
                
                #Continue to expand the space on the line considered until there is at least 'ExpectedWrdsToRight' word(s)
                #Note: We expand the range of characters considered UNIFORMLY on each side of the initital range)
                while wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < ExpectedWrdsToRight+1:
                    
                    #if first iteration just expand the range slightly to see if we are able to find result
                    if itr==2: 
                        
                        #pull in positioin of header +/- 2 char for first try
                        Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr] 
                        
                        #reset itr for start of all subsequent loops to be 1/20'th the total length of the line being considered
                        itr= int(round(Length / ExpansionFactor,0)) 
            
                    #expand the region considered out by a factor of 1/20th the total characters in the line until at least 2 words are present 
                    Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr]
                    itr= itr + int(round(Length/ExpansionFactor,0))
            else: 
                 for i in range(0,20,2):
                        if wordCount(stripwhite(nextstring[int(j[1][0][0])-i:j[1][0][1]+i])) <= 1:
                            Below_Text = nextstring[int(j[1][0][0])-i:j[1][0][1]+i] #pull in positioin of header +/- 2 char for first try
                            print(Below_Text)
                        
                
            #Loop through all matches and extract the associated PO Number 
            for match in finditer(ValidationTest, Below_Text):
                    print(match.group())
                    EndRes = stripwhite(match.group())

#print the end result
EndRes


### Parsing Table Objects 
#### Note: Parsing Tables must be set up  for each company, or set up for groups of companies with the exact same table formating.  
#### Script Built For: Grey Matters Inc.

In [18]:
### 1.Find Start Location / Row of Table 
##### Use regex rules searching for presence of column names associated with the table

#NOTE: TEST contains all matches for the element in question
#identify start and end locaiton of string found for each line in po 
test ={}
list1 = []

#for every line in the PO 
    # apply regex pattern associated with attribute in question 
    # and for each pattern match found in line i generate locational information 
for i in range(len(c)):
       
    #check for match in each line of the rule associated with "TABLE" key
    for match in finditer(LookupTable['TABLE'], c[i]):
        
        #output nested list with tuple of locational information about the match 
        #create a 3 element tuple(line #, where match was found, what match was) with the information on locaiton of string
        list1.append(([str(i)],[match.span()], [match.group()]))

#save to element of dictonary use the same naming convention in the new list
test[LookupTable['TABLE']] = list1

### 2.Identify Start / End location of each header in the table
##### Note: Table Column widths are consistent, so we can pre-define the column width associated with each header
#####       to improve accuracy(at the cost of flexibiilty) if needed 
for i in test.keys():
    for j in test[i]:
        
        #header line (actual string of text where match was found)
        line = c[int(j[0][0])]
        
        #split line into groups of words (seperated by like 4 or 5 spaces) 
        split = line.split('   ')
        
        #remove blank elemnts from list
        split = list(filter(None, split))
        
        #loop through each header value and identify the start and end location of the string 
        StartLoc = []
        for z in range(len(split)):
            
            #header word
            word = split[z]
            
            #start and end loc of each word in the line being parsed
            StartLoc.append([stripwhite(str(word)), int(j[0][0]) ,line.find(str(word)), line.find(str(word))+len(word)])  

### 3.Identify the ending location of the table 
##### Note: Greymatter tables always end after Total or Subtotal line . First we check subtotal because that will 
#####        always appear before total, if subtotal is not found we revert to finding "Total"

#Find Ending location of Table (GreyMatter: either line above subtotal or line above total)
list2 = []
test2 = {}

#Indicators 
Grey1 = None
Grey2 = None 

#for each row in doc c
for i in range(len(c)):
    
    #First look for SUBTOTAL becuase it will always be first(if its there)
    for match in finditer(LookupTable['GREYTABLEEND1'], c[i]): 
        if match is None: #exit loop if no match 
            break    
            
        #output nested list with tuple of locational information about the match 
        list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
        
        #indicator (indicates subtotal found)
        Grey1 = 1
        
for i in range(len(c)): 
    
    #is None: #If you dont find subtotal look for TOTAL 
    if list2 ==[]:
        for match in finditer(LookupTable['GREYTABLEEND2'], c[i]):
            #output nested list with tuple of locational information about the match 
            list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
            #indicator 
            Grey2 = 1
        
        
#assign object to dictonary? 
if Grey1==1:
    #test2[list(LookupTable.keys())[3]] = list2
    test2['GREYTABLEEND1'] = list2
    
    endvar = stripwhite(test2['GREYTABLEEND1'][0][2][0])
    print(endvar)
    print("SubTotal Found")
    
if Grey2==1: 
    test2['GREYTABLEEND2'] = list2
    endvar = stripwhite(test2['GREYTABLEEND2'][0][2][0])
    print(endvar)
    print("Total Found")


#Begining / Ending Line of table 
TableEnd = int(list2[0][0][0])
TableBeg = int(list1[0][0][0])
TableRange = TableEnd - TableBeg

### 4.Option2 (Leverage Static position of table information in regards to column width)
##### Note: this enables retrieval of all information for each column instead of just the
#####       rows associated with the start line for a new item(the rows with the pricing infomration) 
#empty list for results
TableInfo = []

pdframe = None
#loop through each line of the table and extract information 

if endvar =='Subtotal':
    
    #create empty vector for each column 
    Line = []
    Quantity=[]
    Item=[]
    Description=[]
    UnitPrice =[]
    LineTotal =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            Line.append((line,Thisline[:7]))
            Quantity.append((line,Thisline[8:19]))
            Item.append((line,Thisline[20:32]))
            Description.append((line,stripwhite(Thisline[33:55])))
            UnitPrice.append((line,Thisline[75:105]))
            LineTotal.append((line,Thisline[106:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = UnitPrice, columns =["Index","UnitPrice"])
    LineTotal2 = pd.DataFrame(data = LineTotal, columns =["Index","LineTotal"])

    #merge columns back together into final table 
    result = pd.merge(left =Line2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Item2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = Description2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = UnitPrice2, how="left", on="Index")
    result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result5)

    
if endvar =='Total':
    #create empty vector for each column 
    Item=[]
    Quantity=[]
    Description=[]
    Rate =[]
    Amount =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            #variable by comapny / client need
            Item.append((line,Thisline[:21]))
            Quantity.append((line,Thisline[22:36]))
            Description.append((line,stripwhite(Thisline[37:95])))
            Rate.append((line,Thisline[96:115]))
            Amount.append((line,Thisline[116:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    #Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = Rate, columns =["Index","Rate"])
    LineTotal2 = pd.DataFrame(data = Amount, columns =["Index","Amount"])

    #merge columns back together into final table 
    result = pd.merge(left =Item2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Description2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = UnitPrice2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = LineTotal2, how="left", on="Index")
    #result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result4)
    


{re.compile('Item #+( +PARTICULAR)(.*)|\\s+ITEM+( +P. REQ.)(.*)|\\s+ITEM+( +QTY)+( +UM)(.*)|^Item+( +Material/Service)+( Description)(.*)|\\s+Line+( +Quantity)+( +Item)(.*)|\\s+LN+( +ORDER)+( +QTY)(.*)|^Item+( +Q): []}


## Functions 

In [4]:

##Function to determine if pdf is searchable 
def IsPDFSearchable(path): 
    
    #determine if pdf is searchable 
    import os, re
    
    #path is the directory with the files, other 2 are the names of the files you will store your lists in
    path = path
    
    #use this if u want to actually create a file holding all pdfs that are searchable vs not 
    #files_with_text = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//files_with_text.txt", 'a')
    #image_only_files = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//image_only_files.txt", 'a')
    
    files_with_text = []
    image_only_files = []
    
    #have os make a list of all files in that dir for a loop
    filelist = os.listdir(path)
    
    #compile regular expression that matches "Font"
    mysearch = re.compile(r'.*Font.*', re.DOTALL) #.decode("utf-8")
    
    #loop over all files in the directory, open them in binary ('rb'), search that binary for "Font"
    #if they have "Font" they have text, if not they don't
    #(pdf does something to understand the Font type and uses this word every time the pdf contains text)
    for pdf in filelist:
        openable_file = os.path.join(path, pdf)
        cat_file = open(openable_file, 'rb')
        usable_cat_file = cat_file.read()
        #print usable_cat_file
        if mysearch.match(str(usable_cat_file)):
            #files_with_text.write(pdf + '\n') #use if u are storing info in file
            files_with_text.append(pdf)
        else:
            #image_only_files.write(pdf + '\n') #use if u are storing info in file 
            image_only_files.append(pdf)
    
    #use if storing pdf names in files
    #close files holding searchable vs unsearchable pdfs
    #files_with_text.close()
    #image_only_files.close()
    return files_with_text, image_only_files


##https://www.freebsd.org/cgi/man.cgi?query=pdftotext&apropos=0&sektion=0&manpath=FreeBSD+6.2-RELEASE+and+Ports&format=html
#http://www.pdftron.com/assets/pdfs/support/PDFTron_PDF2Text_User_Manual.pdf
#options not used 
##'-remove_hidden_text',
##'-remove_invisible_text'
## pdf2text -c 150,600,250,700 license.pdf -a 1 #extract text from a speicific region 
def get_bbox_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-bbox-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, stdout=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path
    
#return tables 
#return bounding boxes
#for i in range(len(SearchablePDFPaths)):
#    bboxpath = croppath+str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","_BBox.txt")
#    pdfpath = SearchablePDFPaths[i]
#    bboxtextpaths.append(get_bbox_file(pdfpath, bboxpath))

#extract only bounding box information
#lines = []
#for i in range(len(bboxtextpaths)):
#    #lines.append([ line for line in open(bboxtextpaths[i]) if 'block' in line])
#    lines.append([ line.replace("</block>\n","").replace("<block","").replace(">\n","").replace(",","") for line in open(bboxtextpaths[i]) if 'block' in line])

def get_text_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, st=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path

#get files
def getfiles(path, extension):
    items = os.listdir(path)
    newlist = []
    for names in items:
        if names.endswith(extension):
            newlist.append(names)
    return newlist


def isPhoneNumber(text, chars = 11):
    if len(text) != chars:
        return False
    
    for i in range(0, 2):
        if not text[i].isdecimal():
            return False
        #if text[3] != '-':
        #    return False
    for i in range(3, 6):
        if not text[i].isdecimal():
            return False
        #if text[7] != '-':
        #    return False
    for i in range(chars,11):
        if not text[i].isdecimal():
            return False
    return text


def validate(email): 
    match=re.search(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9]+\.[a-zA-Z0-9.]*\.*[com|org|edu]{3}$)",email)
    if match:
        return 'Valid email.'
    else:
        return 'Invalid email.'
    

#count words in string 
def wordCount(mystring):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == " ":  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  


#count words in string 
def wordCountDelim(mystring,delim=" "):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == delim:  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  
    
#remove all white space from string (only use to count total words in line)
def stripwhite(string):

    string = string.rstrip().lstrip() # Remove all extra spaces at the start and at the end of the string
    while "  " in string: # While  there are 2 spaces beetwen words in our string...
        string = string.replace("  ", " ") # ... replace them by one space!
    return(string)

#ID actual start and end position of "Ship To" 
def FindLoc(dic = test,delim='   '):
    import operator
    StartLoc = []
    
    for i in dic.keys():
        for j in test[i]:

        #header line (actual string of text where match was found)
            line = c[int(j[0][0])]

        #split line into groups of words (seperated by like 4 or 5 spaces) 
            split = line.split(delim)

        #remove blank elemnts from list
            split = list(filter(None, split))

        #loop through each header value and identify the start and end location of the string 
            StartLoc = []
            for z in range(len(split)):
            #header word
                word = split[z]
            #start and end loc of each word in the line being parsed
                StartLoc.append([stripwhite(str(word)), int(j[0][0]) ,line.find(str(word)), line.find(str(word))+len(word)])   
            
            #break out of script after first match is processed
            break
        return(StartLoc)   

NameError: name 'test' is not defined